In [ ]:
import numpy as np
import pandas as pd
import glob, h5py
from scipy.io import loadmat

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.gridspec import GridSpec

from impdar.lib.load import load

from netCDF4 import Dataset
import geopandas as gpd

In [ ]:
### Load the processed reflectivity dataframe ###

df = pd.read_csv('../../proc/Processed_Reflectivity.csv')

df_slw = df[(df['Flight Day']==20131227) & (df['Segment'] == 12)]
rclip = -4
lclip = len(df_slw) - len(dat.dist) + rclip
r_rel1 = df_slw['Relative Reflectivity'][lclip:rclip].values

df_gz = df[(df['Flight Day']==20131219) & (df['Segment'] == 2) & (df['Frame'] == 8)]
r_rel2 = df_gz['Relative Reflectivity'].values

In [ ]:
### Load the data as an ImpDAR object ###
dat = load('mcords_mat','../../data/radar/CSARP_standard/Data_img_02_20131227_12_001.mat')[0]
dat.get_projected_coords('epsg:3031')    

### Get the CReSIS surface and bed pick times ###
layers = loadmat('../../data/radar/CSARP_layerData/Data_20131227_12_001.mat')['layerData']
b = layers[0][1][0][0][0][0][1][0][0][0][0]
s = layers[0][0][0][0][0][0][1][0][0][0][0]

### Convert from pick time to sample number and then re-search for the peak bed power ###
bsamp = np.zeros_like(b)
ssamp = np.zeros_like(b)
p = np.empty_like(b)
for trace in range(dat.tnum):
    if np.isnan(b[trace]):
        bsamp[trace] = np.nan
        p[trace] = np.nan
    else:
        b_idx = np.argmin(abs(b[trace]-dat.travel_time*1e-6))
        bsamp[trace] = b_idx
        p[trace] = np.max(dat.data[b_idx-40:b_idx+40,trace])
        
    ### Reset the trace so that the first sample is at the ice surface rather than the airplane (for plotting) ###
    if np.isnan(s[trace]):
        s_idx = 1
        ssamp[trace] = np.nan
    else:
        s_idx = np.argmin(abs(s[trace]-dat.travel_time*1e-6))
        ssamp[trace] = s_idx
    dat.data[:-s_idx,trace] = dat.data[s_idx:,trace]
    dat.data[-s_idx:,trace] = np.nan
    
### Crop the beginning and end from the image ###
dat.hcrop(3238,'right')
dat.hcrop(600,'left')
p = p[599:3237]
b = b[599:3237]
s = s[599:3237]

### Adjust time and depth vectors for updated array ###
dat.travel_time -= dat.travel_time[0]
depth = dat.travel_time*169/2.

### Linear gain for plotting ###
dat.data += np.transpose(np.tile(np.linspace(0,20,dat.snum),(dat.tnum,1)))

In [ ]:
dat2 = load('mcords_mat','../../data/radar/CSARP_standard/Data_img_02_20131219_02_008.mat')[0]
dat2.get_projected_coords('epsg:3031')    

layers = loadmat('../../data/radar/CSARP_layerData/Data_20131219_02_008.mat')['layerData']
b = layers[0][1][0][0][0][0][1][0][0][0][0]
s = layers[0][0][0][0][0][0][1][0][0][0][0]
bsamp2 = np.zeros_like(b)
ssamp2 = np.zeros_like(b)
p2 = np.empty_like(b)
for trace in range(dat2.tnum):
    if np.isnan(b[trace]):
        bsamp2[trace] = np.nan
        p2[trace] = np.nan
    else:
        b_idx = np.argmin(abs(b[trace]-dat2.travel_time*1e-6))
        bsamp2[trace] = b_idx
        p2[trace] = np.max(dat2.data[b_idx-40:b_idx+40,trace])

    ### Reset the trace so that the first sample is at the ice surface rather than the airplane (for plotting) ###
    if np.isnan(s[trace]):
        s_idx = 1
        ssamp2[trace] = np.nan
    else:
        s_idx = np.argmin(abs(s[trace]-dat2.travel_time*1e-6))
        ssamp2[trace] = s_idx
    
    dat2.data[:-s_idx,trace] = dat2.data[s_idx:,trace]
    dat2.data[-s_idx:,trace] = np.nan

### Adjust time and depth vectors for updated array ###
dat2.travel_time -= dat2.travel_time[0]
depth2 = dat2.travel_time*169/2.

dat2.elev = np.nan*np.ones(dat.tnum)
dat2.reverse()
dat2.hcrop(1500,'right')

r_rel2 = r_rel2[::-1][:1499]

In [ ]:
datafold = '/Users/bhills/Desktop/Amphitryon/continental-data-products/' #os.getenv('DATAHOME')

# MODIS Mosaic of Antarctica
moa = '/Users/benhills/Desktop/moa125_2009_hp1_v02.0.nc'
nc = Dataset(moa)
# Crop because we don't need the full continent
moa_x = nc['x'][22000:25000]
moa_y = nc['y'][17000:20000]
moa_z = nc['z'][17000:20000,22000:25000]

In [ ]:
# import subglacial lake outlines (Siegfried & Fricker, 2018)
h5f = h5py.File('../../data/outlines/test.h5', 'r')
lake = 'WhillansSubglacialLake'
x = np.squeeze(h5f[lake]['x'][:])
y = np.squeeze(h5f[lake]['y'][:])

### 
slw_dat = pd.read_csv('../../data/is/slw_elevs.dat')
slw_y = slw_dat['#num month (since 2010)'].values
slw_in = slw_dat['mean elev IN'].values
slw_out = slw_dat['mean elev OUT'].values

# Scripps Grounding Line
gl = '../../data/outlines/grounding-line/scripps/scripps_antarctica_polygons_v1.shp' 
gl_df = gpd.read_file(gl)

In [ ]:
%matplotlib inline

plt.figure(figsize=(10,8))
gs = GridSpec(4,11)

### Panel A - Map View ###

ax1 = plt.subplot(gs[0,:2])
ax1.pcolormesh(moa_x/1000.,moa_y/1000.,moa_z,vmin=15000,vmax=17000,cmap='Greys_r',zorder=-1,rasterized=True)
plt.scatter(df['X']/1000.,df['Y']/1000.,c=df['Relative Reflectivity'],zorder=1,cmap='magma',s=.05,vmin=-20,vmax=20,rasterized=True)
plt.fill_between(np.linspace(-1e8,1e8,10),-1e8,1e8,color='w',alpha=0.5)
plt.scatter(dat.x_coord/1000.,dat.y_coord/1000.,c=r_rel1,s=.5,zorder=3,cmap='magma',vmin=-20,vmax=20,rasterized=True)
plt.plot(x/1000.,y/1000.,c='steelblue',zorder=2)
plt.fill(x/1000.,y/1000.,c='cyan',zorder=0)
plt.axis('equal')
plt.xlim(-290,-247)
plt.ylim(-570,-545)
ax1.xaxis.tick_top()
ax1.xaxis.set_label_position("top")
plt.ylabel('PS Northing (km)')
plt.xlabel('PS Easting (km)')
ax1.text(0.125,.8,'a',transform=ax1.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))

### Panel B - Ice Surface through Time ###

radar_decyr = 2013+361/365.
ax2 = plt.subplot(gs[0,2:6])
plt.plot(slw_y,slw_in-slw_out,'k-o',mfc='w',ms=3.5)
plt.arrow(radar_decyr,1.5,0,-.5,color='k',width=0.025)
plt.arrow(2013+25/365.,.5,0,-.5,color='k',width=0.025)
ax2.xaxis.tick_top()
ax2.xaxis.set_label_position("top")
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ax2.text(0.05,.8,'b',transform=ax2.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))
plt.ylabel('Surf. Anomaly (m)')

### Panel C - Radar Image ###

ax3 = plt.subplot(gs[1:3,:6])
ax3.tick_params(labelbottom=False)
plt.imshow(dat.data,aspect='auto',vmin=-152,vmax=-100,cmap='magma',
          extent=[0,np.max(dat.dist),max(depth),min(depth)],rasterized=True)
plt.ylim(900,0)
plt.ylabel('Depth (m)')
ax3.text(0.035,.9,'c',transform=ax3.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))
# Plot boxes for Figure 3 indicators
x1,x2 = dat.dist[350],dat.dist[750]
y1,y2 = (dat.travel_time[2150])*169/2.,(dat.travel_time[2500])*169/2.
ax3.plot([x1,x1,x2,x2,x1],[y2,y1,y1,y2,y2],c='w')
x1,x2 = dat.dist[1700],dat.dist[2100]
y1,y2 = (dat.travel_time[2000])*169/2.,(dat.travel_time[2350])*169/2.
ax3.plot([x1,x1,x2,x2,x1],[y2,y1,y1,y2,y2],c='w')
x1,x2 = dat.dist[2425],dat.dist[-1]
ax3.plot([x1,x1,x2,x2,x1],[y2,y1,y1,y2,y2],c='w')

### Panel D - Bed Echo Power along Chosen Radar Line ###

ax4 = plt.subplot(gs[3,:6],sharex=ax3)
plt.scatter(dat.dist,r_rel1,c=r_rel1,vmin=-20,vmax=20,s=2,rasterized=True,cmap='magma')
plt.ylabel('Rel. Reflectivity (dB)')
plt.xlabel('Distance (km)')
ax4.text(0.035,.8,'d',transform=ax4.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))
plt.ylim(-30,30)

### Panel E -  ###

ax5 = plt.subplot(gs[0,8:10])
ax5.pcolormesh(moa_x/1000.,moa_y/1000.,moa_z,vmin=15000,vmax=17000,cmap='Greys_r',zorder=-1,rasterized=True)
plt.plot(np.array(gl_df['geometry'][1010].exterior.coords.xy[0])/1000.,np.array(gl_df['geometry'][1010].exterior.coords.xy[1])/1000.,c='w')
plt.scatter(df['X']/1000.,df['Y']/1000.,c=df['Relative Reflectivity'],zorder=1,cmap='magma',s=.05,vmin=-20,vmax=20,rasterized=True)
plt.fill_between(np.linspace(-1e8,1e8,10),-1e8,1e8,color='w',alpha=0.5)
plt.scatter(dat2.x_coord/1000.,dat2.y_coord/1000.,c=r_rel2,s=.5,zorder=3,cmap='magma',vmin=-20,vmax=20,rasterized=True)
plt.axis('equal')
plt.xlim(-185,-140)
plt.ylim(-640,-610)
ax5.xaxis.tick_top()
ax5.xaxis.set_label_position("top")
ax5.yaxis.tick_right()
ax5.yaxis.set_label_position("right")
plt.ylabel('PS Northing (km)')
plt.xlabel('PS Easting (km)')
ax5.text(0.125,.8,'e',transform=ax5.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))

### Panel F - Bed Echo Power Histograms for Entire Survey Area ###

ax6 = plt.subplot(gs[1:3,6:10])
ax6.tick_params(labelleft=False,labelbottom=False)
plt.imshow(dat2.data,aspect='auto',vmin=-152,vmax=-100,cmap='magma',
          extent=[0,np.max(dat2.dist),max(depth2),min(depth2)],rasterized=True)
plt.ylim(900,0)
ax6.text(0.035,.9,'f',transform=ax6.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))
# Plot boxes for Figure 3 indicators
x1,x2 = dat2.dist[11],dat2.dist[3331-3070+250]
y1,y2 = (dat2.travel_time[2000])*169/2.,(dat2.travel_time[2350])*169/2.
ax6.plot([x1,x1,x2,x2,x1],[y2,y1,y1,y2,y2],c='w')
x1,x2 = dat2.dist[3331-2600],dat2.dist[3331-2200]
y1,y2 = (dat2.travel_time[2000])*169/2.,(dat2.travel_time[2350])*169/2.
ax6.plot([x1,x1,x2,x2,x1],[y2,y1,y1,y2,y2],c='w')

### Panel G - Bed Echo Power Histograms for Entire Survey Area ###

ax7 = plt.subplot(gs[3,6:10],sharex=ax6)
ax7.tick_params(labelleft=False)
plt.scatter(dat2.dist,r_rel2,c=r_rel2,vmin=-20,vmax=20,s=2,rasterized=True,cmap='magma')
plt.xlabel('Distance (km)')
ax7.text(0.035,.8,'g',transform=ax7.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))
plt.ylim(-30,30)

### Panel E - Bed Echo Power Histograms for Entire Survey Area ###

ax8 = plt.subplot(gs[3,10],sharey=ax4)
ax8.tick_params(labelleft=False)
bins = np.arange(-50,50,.25)
plt.hist(df[(df['Grounded']==True) & (df['Lake']==False)]['Relative Reflectivity'],bins=bins,
         orientation='horizontal',rasterized=True,alpha=1,color='grey'); #cm.magma((20-3.8)/40.));
plt.hist(df[df['Grounded']==False]['Relative Reflectivity'],bins=bins,
         orientation='horizontal',rasterized=True,alpha=1,color='lightgrey'); #cm.magma(28.38/40.));
bins = np.arange(-50,50,.5)
plt.hist(df[df['Lake']==True]['Relative Reflectivity'],bins=bins,
         orientation='horizontal',rasterized=True,alpha=1,color='cyan'); #cm.magma((20-13.7)/40.),);
plt.xlabel('# Traces')
ax8.text(0.5,.8,'h',transform=ax8.transAxes,fontweight='bold',fontsize=12,
         bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=1.5))


### Cleanup and Save ###
#plt.tight_layout()
plt.savefig('fig2-slw-image.pdf',dpi=300)